<h1>Grafo delle co-authorship tra gli articoli del corso di matematica</h1>
<p>Dal dataset degli articoli di matematica vengono filtrati solamente quelli dei professori che attualmente sono nella lista del dipartimento di informatica.</p>

In [47]:
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import re
from collections import Counter

#from src.grafico_informatica_prima_versione import df_professori_articoli

<h2>Caricamento dei dati</h2>

<h3>Dataset degli articoli di fisica</h3>
<p>In formato csv e convertito a dataframe</p>

In [48]:
# Carico il dataset
df_articoli_matematica = pd.read_csv("../data/matematica.csv")

<h3>Creazione della lista dei professori di informatica</h3>
<p>I nomi sono stati copiati dal sito dell'università e incollati in un file di testo.</p>

In [49]:
# Carico la lista dei professori dal file txt
with open("../data/professori_dipartimento_inf.txt", "r", encoding="utf-8") as f:
    professori_informatica = [riga.strip() for riga in f]

with open("../data/professori_dipartimento_mat.txt", "r", encoding="utf-8") as f:
    professori_matematica = [riga.strip() for riga in f]

def normalize_prof(name):
    cognome, nome = [x.strip() for x in name.split(',')]
    return f"{cognome} {nome}"

def normalize_list(lista_professori):
    lista_professori = set(normalize_prof(n) for n in lista_professori)
    return list(lista_professori)


professori_informatica = normalize_list(professori_informatica)
professori_matematica = normalize_list(professori_matematica)

print(professori_informatica)

print("Prof di matematica: \n", professori_matematica)

['Scagnetto Ivan', 'Omero Paolo', 'Maddalena Eddy', 'Liessi Davide', 'Piazza Carla', 'Coppola Paolo', 'Mirolo Claudio', 'Vermiglio Rossana', 'Brajnik Giorgio', 'Roitero Kevin', 'Cimatti Alessandro', 'Serra Giuseppe', 'Fogolari Federico', 'Policriti Alberto', 'Buttussi Fabio', 'Puppis Gabriele', 'Mameli Valentina', 'Comini Marco', 'Bellettini Giovanni', 'Della Mea Vincenzo', 'Lancia Giuseppe', 'Fontana Federico', 'Vidoni Paolo', 'Romani Giulio', 'Geatti Luca', 'Chittaro Luca', 'Burigat Stefano', 'Mizzaro Stefano', 'Riccio Vincenzo', 'Lenisa Marina', 'Di Gianantonio Pietro', 'Dovier Agostino', 'Miculan Marino', 'Della Monica Dario', 'Dimonte Vincenzo', 'Dattolo Antonina', 'Franceschet Massimo', 'Micheloni Christian', 'Montanari Angelo', 'Bozzo Enrico', 'Fortuna Sara', 'D’Agostino Giovanna', 'Drioli Carlo', 'Fusiello Andrea', 'Ritacco Ettore', 'Pace Luigi', 'Formisano Andrea']
Prof di matematica: 
 ['Giannerini Simone', 'Marcone Alberto', 'Liessi Davide', 'Piazza Carla', "D'Agostino Giova

<h2>Funzione per estrarre gli autori dalla relativa colonna del dataframe</h2>
<p>Gli autori sono memorizzati come: "Cognome, Nome (ID); ..."<br>
Quindi vogliamo trasformarli in "Cognome Nome"
</p>

In [50]:
def pulisci_autori(author_string):
    if pd.isna(author_string):
        return []
    
    autori = []
    for a in author_string.split(";"):
        a = a.strip()
        # prendo la parte prima di "("
        a = a.split("(")[0].strip()
        
        # separo Cognome, Nome
        if "," in a:
            cognome, nome = a.split(",", 1)
            autori.append(f"{cognome.strip()} {nome.strip()}")
        else:
            autori.append(a)
    return autori

<h2>Creazione del grafo</h2>
<p>Viene anche salvato in formato pickle in modo tale che possa essere utilizzato in altri notebook</p>

In [51]:
G_prof_informatici = nx.Graph()
G_prof_matematica = nx.Graph()
G_prof_fisica= nx.Graph()

def creaGrafoProf(G,lista, path_pickle):
    for autori_string in df_articoli_matematica["Author full names"]:
        lista_autori = pulisci_autori(autori_string)

        # Filtra solo i professori del dipartimento
        autori_prof = [a for a in lista_autori if a in lista]

        # Aggiungi edge tra tutte le coppie di professori che hanno co-firmato
        for i in range(len(autori_prof)):
            for j in range(i + 1, len(autori_prof)):
                a1, a2 = autori_prof[i], autori_prof[j]

                if G.has_edge(a1, a2):
                    G[a1][a2]["weight"] += 1
                else:
                    G.add_edge(a1, a2, weight=1)
    with open(path_pickle, "wb") as f:
        pickle.dump(G, f)

creaGrafoProf(G_prof_informatici,professori_informatica ,"grafi_pickle/articoli_matematica/G_prof_informatici.pkl")
creaGrafoProf(G_prof_matematica, professori_matematica,"./grafi_pickle/articoli_matematica/G_prof_matematica.pkl")

<h3>Dati del grafo</h3>
<ul>
    <li>Numero di nodi (professori coinvolti)</li>
    <li>Numero di archi (co-authorship)</li>
    <li>Per ogni professore i relativi collaboratori</li>
</ul>

In [52]:
print("Numero di nodi (professori coinvolti):", G_prof_matematica.number_of_nodes())
print("Numero di archi (co-autorship):", G_prof_matematica.number_of_edges())

# Mostra i vicini di un professore, se vuoi
for n in G_prof_matematica.nodes():
    print(n, "co-autore con:", list(G_prof_matematica.neighbors(n)))


Numero di nodi (professori coinvolti): 15
Numero di archi (co-autorship): 13
D'Agostino Giovanna co-autore con: ['Policriti Alberto', 'Marcone Alberto']
Policriti Alberto co-autore con: ["D'Agostino Giovanna", 'Piazza Carla', 'Marcone Alberto']
Breda Dimitri co-autore con: ['Liessi Davide', 'Vermiglio Rossana']
Liessi Davide co-autore con: ['Breda Dimitri', 'Vermiglio Rossana']
Corvaja Pietro co-autore con: ['Zucconi Francesco', 'Piazza Carla']
Zucconi Francesco co-autore con: ['Corvaja Pietro', 'De Poi Pietro']
Piazza Carla co-autore con: ['Policriti Alberto', 'Corvaja Pietro']
Vermiglio Rossana co-autore con: ['Breda Dimitri', 'Liessi Davide']
Feltrin Guglielmo co-autore con: ['Gidoni Paolo']
Gidoni Paolo co-autore con: ['Feltrin Guglielmo']
Fasino Dario co-autore con: ['Rinaldi Franca']
Rinaldi Franca co-autore con: ['Fasino Dario', 'Blanchini Franco']
Marcone Alberto co-autore con: ["D'Agostino Giovanna", 'Policriti Alberto']
De Poi Pietro co-autore con: ['Zucconi Francesco']
Blanc

<h3>Visualizzazione del grafo tramite libreria Pyvis</h3>
<p>Il file viene salvato in formato .html nella relativa cartella</p>

In [53]:
# Crea un grafo pyvis
def creaGrafo(G,output_path):
    net = Network(height="750px", width="100%", bgcolor="black", font_color="white", notebook=True, cdn_resources="remote", filter_menu=True)

    # Forza fisica per un layout più bello
    net.force_atlas_2based()

    # Aggiungi nodi e archi da NetworkX a PyVis
    for node in G.nodes():
        net.add_node(
            node,
            label=node,
            title=node,
            color="#4A90E2",   # colore nodi
            shape="dot",
            size=15
        )

    for u, v, data in G.edges(data=True):
        weight = data.get("weight", 1)
        net.add_edge(
            u,
            v,
            value=weight,
            title=f"Co-authored papers: {weight}"
        )
    # Salva in HTML
    net.write_html(output_path)
    print("File generato:", output_path)
    return net
output_path_informatica= "../html/grafi_co-authorship/artcoli_matematica/professori-informatica.html"
creaGrafo(G_prof_informatici, output_path_informatica).show(output_path_informatica)




File generato: ../html/grafi_co-authorship/artcoli_matematica/professori-informatica.html
../html/grafi_co-authorship/artcoli_matematica/professori-informatica.html


In [54]:
output_path_matematica= "../html/grafi_co-authorship/artcoli_matematica/professori-matematica.html"
creaGrafo(G_prof_matematica, output_path_matematica).show(output_path_matematica)

File generato: ../html/grafi_co-authorship/artcoli_matematica/professori-matematica.html
../html/grafi_co-authorship/artcoli_matematica/professori-matematica.html
